In [1]:
import csv
import pathlib
import json
import numpy as np

In [2]:
file_list = ['train', 'test','dev']
subject = ['Movies_1', 'Movies_2', 'Movies_3']
user = {}
intents = set()

In [3]:
def discover_intents(jsonfile, file_name):
    for item in jsonfile:
        if len(item['services']) > 1 or not np.any(np.in1d(subject, item['services'])):
            continue
        for turn in item['turns']:
            if turn['speaker'] == 'SYSTEM':
                continue
            text = turn['utterance']
            intent = 'other'
            for frame in turn['frames']:
                if frame['service'] not in subject:
                    continue
                for action in frame['actions']:
                    intent = action['act']
                    if text not in user:
                        user[text] = set()
                        user[text].add(intent.upper())
                        intents.add(intent)
        

In [4]:
if __name__ == '__main__':
    for file_name in file_list:
        for path in pathlib.Path(f'../dstc8-schema-guided-dialogue/{file_name}').iterdir():
             if path.is_file() and not path.match('*schema.json'):
                    file = open(path, 'r')
                    json_file = json.loads(file.read())
                    discover_intents(json_file,path)
                    file.close()
        f = open(f'./classification/{file_name}_dataset.csv', 'w')
        writer = csv.writer(f)
        writer.writerow(['Sentence', 'Intent'])
        for sent, intent_list in user.items():
            for intent in intent_list:
                writer.writerow([sent, intent])
        f.close()
    for _intent in intents:
        print(_intent)
                    

REQUEST
NEGATE
NEGATE_INTENT
SELECT
THANK_YOU
REQUEST_ALTS
INFORM_INTENT
INFORM
AFFIRM_INTENT
AFFIRM
